Imports

In [12]:
import smtplib, pyautogui

from time import sleep
from math import ceil

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager


Initializations

In [13]:
delay = 1

email_address = "ENTER EMAIL ADDRESS"
email_password = "ENTER PASSWORD"

subject_file = open("subject.txt", "r")
subject = subject_file.read()

message_file = open("message.txt", "r")
message = message_file.read()

url = "https://mca.gov.in/content/mca/global/en/data-and-reports/company-llp-info/incorporated-closed-month.html"
files_needed = 30  # set number of files to be downloaded


Initialzing Web Driver

In [14]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)
# pyautogui.hotkey("win", "d")
# pyautogui.hotkey("alt", "tab")
# pyautogui.hotkey("alt", "tab")


Determing files to be downloaded

In [15]:
driver.get(url)
driver.find_element(by=By.CLASS_NAME, value="expand-desk").click()
month = driver.find_element(
    by=By.XPATH,
    value="/html/body/div/div/div[2]/div/div[3]/div/div/div[2]/div[2]/div/div/table[1]/tbody/tr[2]/td/div/div/div/div/div[7]/table/tbody/tr[1]/td[2]",
).text

year = driver.find_element(
    by=By.XPATH,
    value="/html/body/div/div/div[2]/div/div[3]/div/div/div[2]/div[2]/div/div/table[1]/tbody/tr[2]/td/div/div/div/div/div[7]/table/tbody/tr[1]/td[3]",
).text

print(f"Latest Files Avilable are of: {month}/{year}")


Latest Files Avilable are of: August/2022


Downloading the files

In [16]:
loops = ceil(files_needed / 5)
counter = 0
for loop in range(loops):
    if loop != 0:
        driver.find_element(by=By.CLASS_NAME, value="next").click()
    download_element_list = driver.find_elements(by=By.CLASS_NAME, value="doc-link")
    download_element_list = download_element_list[0:5]
    for download_element in download_element_list:
        if counter < files_needed:
            download_element.click()
            counter += 1
            # sleep(3)


Read Excel Sheets

In [ ]:
a = 0
b = 0
wb = xlrd.open_workbook(r"C:\Users\shiva\Downloads\Book1.xls")
sheet = wb.sheet_by_index(0)
sheet.cell_value(0, 0)

for x in range(sheet.ncols):
    column_name = sheet.cell_value(0, x)
    if (column_name.lower()).find("email") != -1 and a == 0:
        email_column = x
        a = 1
    if (column_name.lower()).find("name") != -1 and b == 0:
        name_column = x
        b = 1


Send Emails

In [ ]:
with smtplib.SMTP("smtp.gmail.com") as connection:
    connection.starttls()
    connection.login(user=email_address, password=email_password)

    for i in range(1, sheet.nrows):
        to_email_address = sheet.cell_value(i, email_column)
        company_name = (sheet.cell_value(i, name_column)).title()

        connection.sendmail(
            from_addr=email_address,
            to_addrs=to_email_address,
            msg="Subject:"
            + subject
            + "\n\n"
            + "Hi! hope you are having a wonderful day at "
            + company_name
            + ",\n"
            + message,
        )
        print(f"{str(i)}. Email sent to {company_name} successfully")
        sleep(delay)


Quit Webdriver

In [11]:
driver.quit()
